In [2]:
from ipymlwidgets import Image, ImageAnnotated
import torch
import ipywidgets as W
size = (320,480)

xy = torch.randint(size[0]-10, (1,2))
boxes = torch.cat([xy, xy + 10, torch.zeros_like(xy[:, :1])], dim=-1).float()
#image = Image(torch.rand(3,*size))
image = ImageAnnotated(torch.rand((3,*size)), boxes=boxes)
display(image)


ImageAnnotated(children=(Box(children=(HTML(value='\n<style>\n.image-overlay {\n    position: absolute;\n    t…

In [2]:
print(image.boxes)

tensor([[ 6.0000,  7.0000, 16.0000, 17.0000,  0.0000],
        [20.7000,  9.5000, 20.7000,  9.5000,  0.0000]])


In [1]:

from ipymlwidgets import ImageAnnotated
import torch
import ipywidgets as W

size = (320,320)
image = ImageAnnotated(torch.rand((3,32,32)), display_size=size)
selection_text = W.Text(value="",disabled=True,layout=W.Layout(width=f'100%'))
def update_selection_text(change):
    selection_text.value = str(change["new"])
    
label_text = W.Text(value="", disabled=False, layout=W.Layout(width=f'100%'))

image.observe(update_selection_text, "selection")

xy = torch.randint(image.size[0]-10, (1,2))
boxes = torch.cat([xy, xy + 10, torch.zeros_like(xy[:, :1])], dim=-1).float()
image.boxes = boxes

display(selection_text)
display(image)
display(label_text)

TypeError: ImageAnnotated.__init__() got an unexpected keyword argument 'display_size'

In [3]:
print(image.boxes)
print(image._overlay_box.boxes)

tensor([[ 9.0000, 18.0000, 19.0000, 28.0000,  0.0000],
        [13.2000,  5.9000, 23.1000, 13.0000,  0.0000],
        [ 4.3000,  5.6000,  4.3000,  5.6000,  0.0000]])
tensor([[ 90., 180., 190., 280.,   0.],
        [132.,  59., 231., 130.,   0.],
        [ 43.,  56.,  95., 118.,   0.]])


In [1]:
from ipymlwidgets.widgets.image_ocr import ImageOCR
import torch

size = (32,48)
image = torch.rand((3,size[1],size[0]))
x = torch.randint(size[0]-10, (1,1))
y = torch.randint(size[1]-10, (1,1))
boxes = torch.cat([x, y, x + 10, y + 10], dim=-1).float()
print(boxes)

image_ocr = ImageOCR(image, boxes=boxes, texts=["hello there"])# display_size=(size[0]*10,size[0]*10))
display(image_ocr)

tensor([[11., 19., 21., 29.]])


ImageOCR(children=(Box(children=(HTML(value='\n<style>\n.overlay-image {\n    position: absolute;\n    top: 0;…

In [ ]:
from traitlets import Int, dlink, link, HasTraits

class A(HasTraits):
    x = Int()


class B(HasTraits):
    x = Int()

    def __init__(self, a: A):
        self.a = a
        dlink((self, "x"), (self.a, "x"), self.transform_ba)
        dlink((self.a, "x"), (self, "x"), self.transform_ab)

    def transform_ab(self, value):
        return value + 7
    
    def transform_ba(self, value):
        return value - 7


a = A() 
b = B(a)

print(a.x, b.x)

a.x += 1

print(a.x, b.x)

b.x += 1

print(a.x, b.x)

In [ ]:
from ipymlwidgets.widgets.box_overlay import BoxOverlay
import torch
size = (320,60)
x = torch.randint(size[0]-10, (1,1))
y = torch.randint(size[1]-10, (1,1))
boxes = torch.cat([x, y, x + 10, y + 10], dim=-1).float()

overlay = BoxOverlay(boxes, overlay_size=size)
display(overlay)

#box.observe_mouse_click(print)


In [ ]:
x = torch.empty(0)
print(x.ndim, x.shape)

In [ ]:
print(image.boxes)

In [ ]:

from ipymlwidgets.widgets import Image
import torch

image = Image(torch.rand((3,10,10)))
display(image)
image.observe_mouse_click(print)



In [ ]:
import ipywidgets as widgets
from ipymlwidgets.widgets.image import Image
import torch

class OverlayContainer(widgets.Box):
    """A container that overlays multiple images using CSS."""
    
    def __init__(self, width: int = 640, height: int = 480, **kwargs):
        super().__init__(**kwargs)
        
        # Add CSS for overlay positioning
        self.add_class("overlay-container")
        
        # Set up layout
        self.layout = widgets.Layout(
            width=f"{width}px",
            height=f"{height}px"
        )

# Add CSS to the page
css = """
<style>
.overlay-image {
    position: absolute;
    top: 0;
    left: 0;
}
</style>
"""

size = (640,640 )
# Create container and images
container = OverlayContainer(width=size[0], height=size[1])

image = torch.randn((3,10,10))

image1 = Image(image=torch.ones((3,10,10)) * 200, layers=1)
image2 = Image(image=torch.randn((4,200,200)), layers=1)
image1.add_class("overlay-image")
image2.add_class("overlay-image")

container.children = [widgets.HTML(css), image1, image2]

# Display
display(container)

print(image1.layout)

In [ ]:
import ipywidgets as widgets
from ipymlwidgets.widgets.image import Image
import torch

OVERLAY_CSS = """
<style>
.overlay-image {
    position: absolute;
    top: 0;
    left: 0;
}
</style>
"""
class ImageAnnotated(widgets.Box):
    
    def __init__(self, image, overlay, size : tuple[int,int] = (320,320)):
        self._image_canvas = Image(image)
        self._image_canvas.add_class("overlay-image")
        self._overlay_canvas = Image(overlay)
        self._overlay_canvas.add_class("overlay-image")
        size = (f"{size[0]}px", f"{size[1]}px") if size is not None else ("100%", "auto")
        layout = widgets.Layout(
            width=size[0],
            height=size[1],
        )
        super().__init__([widgets.HTML(OVERLAY_CSS), self._image_canvas, self._overlay_canvas], layout=layout)


image = torch.zeros((3,10,10))
image[0,:,:] = torch.rand((10,10))

overlay = torch.zeros((4,50,60)) 
overlay[-1] = torch.rand((50,60))

#overlay = torch.zeros((4,50,50))

# Create container and images
image = ImageAnnotated(
    image,
    overlay,
)


image._overlay_canvas.observe_mouse_click(print)
image._image_canvas.observe_mouse_click(print)

display(image)